### Load modules

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import KFold

In [3]:
import h5py

### Load train image list

In [4]:
df = pd.read_csv('driver_imgs_list.csv')
# drivers' names
subjects = np.unique(df['subject'])
# classes
classnames = np.unique(df['classname'])

### 5-fold train-val split in terms of drivers: 

In [5]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
subjects_t = []
subjects_v = []
for sub_t, sub_v in kf.split(subjects):
    subjects_t.append(list(subjects[sub_t]))
    subjects_v.append(list(subjects[sub_v]))

split the train image list

In [6]:
def split_list(subjects_t, subjects_v):
    df_t = df.loc[df.subject.isin(subjects_t)]
    df_v = df.loc[df.subject.isin(subjects_v)]
    return df_t, df_v

### original dataset dir.

In [7]:
dataset_dir = '/dataset_dir'
# dataset_dir = '../../dataset_resized'

train_raw_dir = dataset_dir + '/train'
test_dir = dataset_dir + '/test'

### make dirs of train and validation

In [8]:
import shutil
from shutil import *

In [9]:
from tqdm import tqdm

In [10]:
def mkdir_train_val(fold):
    if 'train'+str(fold) in os.listdir():
        shutil.rmtree('train'+str(fold))
    if 'validation'+str(fold) in os.listdir():
        shutil.rmtree('validation'+str(fold))
    for split_name in ['train', 'validation']:
        for cls in classnames:
            os.makedirs(split_name+str(fold) + '/' + cls)

In [11]:
def symlink_train_val(df_t, df_v, fold, train_dir):
    mkdir_train_val(fold)
    for i in tqdm(range(len(df_t))):
        cls = df_t.iloc[i]['classname']
        img = df_t.iloc[i]['img']
        os.symlink(train_dir+'/'+cls+'/'+img, 'train'+str(fold)+'/'+cls+'/'+img)
    for i in tqdm(range(len(df_v))):
        cls = df_v.iloc[i]['classname']
        img = df_v.iloc[i]['img']
        os.symlink(train_dir+'/'+cls+'/'+img, 'validation'+str(fold)+'/'+cls+'/'+img)

In [12]:
def copy_train_val(df_t, df_v, fold, train_dir):
    mkdir_train_val(fold)
    for i in tqdm(range(len(df_t))):
        cls = df_t.iloc[i]['classname']
        img = df_t.iloc[i]['img']
        copyfile(train_dir+'/'+cls+'/'+img, 'train'+str(fold)+'/'+cls+'/'+img)
    for i in tqdm(range(len(df_v))):
        cls = df_v.iloc[i]['classname']
        img = df_v.iloc[i]['img']
        copyfile(train_dir+'/'+cls+'/'+img, 'validation'+str(fold)+'/'+cls+'/'+img)

In [13]:
for fold in range(1):
    df_t, df_v = split_list(subjects_t[fold], subjects_v[fold])
    symlink_train_val(df_t, df_v, fold, train_raw_dir)

100%|██████████| 4978/4978 [00:34<00:00, 142.38it/s]


In [14]:
train_dir = 'train0'
val_dir = 'validation0'

### preprocessing images

In [10]:
# preprocess function of VGG and ResNet50
def preprocess_fn(x):
    # RGB >> BGR
    x = img_to_array(x)
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [11]:
# default image size of VGG and ResNet50
img_width, img_height = 224, 224

In [12]:
batch_size = 32

In [13]:
data_gen = ImageDataGenerator(preprocessing_function=preprocess_fn)

original generator

In [19]:
train_generator = data_gen.flow_from_directory(train_dir, 
                                               target_size=(img_height, img_width), 
                                               class_mode='categorical',
                                               batch_size=batch_size)

Found 17446 images belonging to 10 classes.


In [19]:
val_generator = data_gen.flow_from_directory(val_dir, 
                                             target_size=(img_height, img_width), 
                                             class_mode='categorical',
                                             batch_size=batch_size)

Found 4978 images belonging to 10 classes.


In [14]:
test_generator = data_gen.flow_from_directory(test_dir, 
                                              target_size=(img_height, img_width), 
                                              class_mode=None,
                                              shuffle=False,
                                              batch_size=batch_size)

Found 79726 images belonging to 1 classes.


transfer generator

In [ ]:
train_tf_generator = data_gen.flow_from_directory(train_dir, 
                                                  target_size=(img_height, img_width), 
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=batch_size)

In [ ]:
val_tf_generator = data_gen.flow_from_directory(val_dir,
                                                target_size=(img_height, img_width), 
                                                class_mode='categorical',
                                                shuffle=False,
                                                batch_size=batch_size)

augment generator

In [21]:
data_gen_aug = ImageDataGenerator(rotation_range=30.,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  preprocessing_function=preprocess_fn)

In [22]:
train_aug_generator = data_gen_aug.flow_from_directory(train_dir, 
                                                       target_size=(img_height, img_width), 
                                                       class_mode='categorical',
                                                       batch_size=batch_size)

Found 17446 images belonging to 10 classes.


In [42]:
val_aug_generator = data_gen_aug.flow_from_directory(val_dir, 
                                                     target_size=(img_height, img_width), 
                                                     class_mode='categorical',
                                                     batch_size=batch_size)

Found 4978 images belonging to 10 classes.


In [44]:
test_aug_generator = data_gen_aug.flow_from_directory(test_dir, 
                                                      target_size=(img_height, img_width), 
                                                      class_mode=None,
                                                      shuffle=False,
                                                      batch_size=batch_size)

Found 79726 images belonging to 1 classes.


### Fine-tuning VGG16

In [15]:
from keras.optimizers import *
from keras.callbacks import *

In [16]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

In [17]:
def model_build(MODEL, optimizer, layer_num_fix, top_dropout=False):
    base_model = MODEL(weights='imagenet', include_top=True)
    if top_dropout:
        x = Dropout(0.4, name='top_dropout')(base_model.layers[-2].output)
        x = Dense(10, activation='softmax', name='top_prediction')(x)
    else:
        x = Dense(10, activation='softmax', name='top_prediction')(base_model.layers[-2].output)
        
    model_ft = Model(base_model.input, x, name=MODEL.__name__)
    model_ft.compile(optimizer=optimizer, 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    for layer in model_ft.layers[:layer_num_fix]:
        layer.trainable = False
    return model_ft

In [18]:
model_vgg16_ft = model_build(VGG16, adam, 10, False)

In [27]:
tensorboard_cb = TensorBoard(log_dir='log/vgg16', histogram_freq=0, batch_size=batch_size, write_graph=True, write_images=True)
ckpt_cb = ModelCheckpoint('vgg16_ft0_aug_weights.{epoch:02d}-{val_loss:.3f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True)
stp_cb = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='auto')

In [ ]:
num_epochs = 80

In [ ]:
model_vgg16_ft.fit_generator(train_aug_generator, 
                             50,
                             num_epochs, 
                             validation_data=val_generator, 
                             validation_steps=val_generator.samples//batch_size+1,
                             callbacks=[tensorboard_cb, ckpt_cb])

Epoch 1/80
50/50 [==============================] - 188s - loss: 2.4422 - acc: 0.1113 - val_loss: 2.2570 - val_acc: 0.1687
Epoch 2/80
50/50 [==============================] - 178s - loss: 2.0875 - acc: 0.2200 - val_loss: 1.8026 - val_acc: 0.3823
Epoch 3/80
50/50 [==============================] - 178s - loss: 1.3650 - acc: 0.4769 - val_loss: 1.5695 - val_acc: 0.4516
Epoch 4/80
50/50 [==============================] - 178s - loss: 1.0367 - acc: 0.6069 - val_loss: 1.5054 - val_acc: 0.4994
Epoch 5/80
50/50 [==============================] - 178s - loss: 0.7031 - acc: 0.7288 - val_loss: 1.0649 - val_acc: 0.6181
Epoch 6/80
50/50 [==============================] - 178s - loss: 0.5636 - acc: 0.8106 - val_loss: 0.9989 - val_acc: 0.7091
Epoch 7/80
50/50 [==============================] - 178s - loss: 0.4695 - acc: 0.8400 - val_loss: 0.6067 - val_acc: 0.8092
Epoch 8/80
50/50 [==============================] - 167s - loss: 0.3322 - acc: 0.8869 - val_loss: 0.7312 - val_acc: 0.8029
Epoch 9/80
50/50

50/50 [==============================] - 165s - loss: 0.1048 - acc: 0.9719 - val_loss: 0.9270 - val_acc: 0.7836
Epoch 67/80
50/50 [==============================] - 166s - loss: 0.0790 - acc: 0.9800 - val_loss: 0.7932 - val_acc: 0.8582
Epoch 68/80
50/50 [==============================] - 166s - loss: 0.0745 - acc: 0.9794 - val_loss: 0.9371 - val_acc: 0.8114
Epoch 69/80
50/50 [==============================] - 166s - loss: 0.0649 - acc: 0.9800 - val_loss: 1.1489 - val_acc: 0.8240
Epoch 70/80
50/50 [==============================] - 166s - loss: 0.0770 - acc: 0.9813 - val_loss: 0.6556 - val_acc: 0.8813
Epoch 71/80
50/50 [==============================] - 166s - loss: 0.0550 - acc: 0.9838 - val_loss: 0.9787 - val_acc: 0.8542
Epoch 72/80
50/50 [==============================] - 166s - loss: 0.0432 - acc: 0.9888 - val_loss: 0.7563 - val_acc: 0.8576
Epoch 73/80
50/50 [==============================] - 166s - loss: 0.0486 - acc: 0.9881 - val_loss: 0.6778 - val_acc: 0.8746
Epoch 74/80
50/50 [=

In [19]:
model_vgg16_ft.load_weights('vgg16_ft0_aug_weights.31-0.485.hdf5')

In [20]:
def write_predicts_gen(test_gen, batch_size, sub_fname):
    y_pred = model_vgg16_ft.predict_generator(test_gen, 
                                              test_gen.samples//batch_size+1, 
                                              verbose=1)

    y_pred_df = pd.DataFrame(y_pred, columns=classnames)
    
    img_name = []
    for i, fname in enumerate(test_gen.filenames):
        img_name.append(os.path.basename(fname))

    img_name_df = pd.DataFrame(img_name, columns=['img'])
    predictions = pd.concat([img_name_df, y_pred_df], axis=1)
    print(predictions.shape)
    predictions.to_csv(sub_fname, index=False)
    print('submission file {} is ready.'.format(sub_fname))

In [21]:
write_predicts_gen(test_generator, batch_size, 'submission_vgg16_ft0_aug.csv')

2492/2492 [==============================] - 1263s  
(79726, 11)
submission file submission_vgg16_ft0_aug.csv is ready.
